In [141]:
import pandas as pd
import numpy as np
import video.df as df
from video import vid



In [5]:
path = 'videos/Red Spin.mp4'
vid = vid.Video.from_file(path)
vid_df = df.get_vid_df(path)

In [8]:
#vid.difference(1, 'lightness')

In [10]:
vid_df.head()

hue  lightness  saturation
frame y x                            
0     0 0    0        126         255
        1    0        126         255
        2    0        126         255
        3    0        126         255
        4    0        126         255

In [12]:
edf = df.get_exploration_df(path)

In [13]:
agg_df = df.get_aggregated_df(edf)

In [64]:
edf.head(2)

hue  lightness  saturation  masked_values  seconds
frame y x                                                    
0     0 0    0        126         255              0        0
        1    0        126         255              0        0

In [65]:
agg_df.head(2)

,lightness,hue,saturation,masked_values,light_diff,hue_diff,saturation_diff,mask_diff
frame,,,,,,,,
0,33.5124,0.1416,67.1892,0,NaN,NaN,NaN,NaN
1,32.5144,0.0004,65.4256,0,0.998,0.1412,1.7636,0.0


In [66]:
edf.shape, agg_df.shape

((750000, 5), (300, 8))

In [ ]:
import numpy as np

# Load the vectors into a list
vectors = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]

# Determine the maximum length of the vectors
max_len = max(len(v) for v in vectors)

# Pad the vectors
padded_vectors = []
for v in vectors:
    padded_vector = np.pad(v, (0, max_len - len(v)), 'constant')
    padded_vectors.append(padded_vector)

# Create a mask
mask = []
for v in padded_vectors:
    mask.append([1] * len(v) + [0] * (max_len - len(v)))

# Convert the lists to numpy arrays
padded_vectors = np.array(padded_vectors)
mask = np.array(mask)

# Label the vectors
labels = np.array([0, 1, 0])


In [17]:
import os

# specify the directory path
directory_path = "videos1"

# get the list of filenames in the directory
filenames = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# print the list of filenames
print(filenames)


['hazard.01.mp4', 'hazard.02.mp4', 'safe.04.mp4', 'hazard.03.mp4', 'safe.01.mp4', 'hazard.07.mp4', 'hazard.06.mp4', 'safe.02.mp4', 'hazard.10.mp4', 'hazard.04.mp4', 'hazard.05.mp4', 'safe.03.mp4', 'hidden.05.mp4', 'hazard.08.mp4', 'hazard.09.mp4', 'hidden.04.mp4', 'hidden.06.mp4', 'hidden.03.mp4', 'hidden.02.mp4', 'hidden.01.mp4']


In [18]:
# get the list of filenames in the directory
filenames1 = os.listdir(directory_path)

# print the list of filenames
print(filenames1)

['hazard.01.mp4', 'hazard.02.mp4', 'safe.04.mp4', 'hazard.03.mp4', 'safe.01.mp4', 'hazard.07.mp4', 'hazard.06.mp4', 'safe.02.mp4', 'hazard.10.mp4', 'hazard.04.mp4', 'hazard.05.mp4', 'safe.03.mp4', 'hidden.05.mp4', 'hazard.08.mp4', 'hazard.09.mp4', 'hidden.04.mp4', 'hidden.06.mp4', 'hidden.03.mp4', 'hidden.02.mp4', 'hidden.01.mp4']


In [142]:
import video.reader as r
import cv2 as cv
from video.reader import VideoReader

In [29]:
vid, fps = r.VideoReader.get_vid(path, cv.COLOR_BGR2HLS)

In [30]:
vid.shape

(300, 50, 50, 3)

In [40]:
vid[0][0][1]

array([  0, 126, 255], dtype=uint8)

In [55]:
vid.reshape(-1,3).shape

(750000, 3)

In [72]:
lightness_per_frame = vid.reshape(-1, 3)[:, 1].reshape((-1, 50*50))

In [73]:
av_lightness_per_frame = np.mean(lightness_per_frame,axis=1)

In [74]:
av_lightness_per_frame.shape

(300,)

In [76]:
av_lightness_per_frame[:5]

array([33.5124, 32.5144, 31.4948, 32.4748, 33.5124])

In [78]:
agg_df.lightness[:5]

frame
0    33.5124
1    32.5144
2    31.4948
3    32.4748
4    33.5124
Name: lightness, dtype: float64

In [121]:
shifed_lightness = np.concatenate([np.zeros(1), av_lightness_per_frame[:-1]])

In [113]:
shifed_lightness[0] = np.nan

In [107]:
agg_df.lightness.shift(1)[:5]

frame
0        NaN
1    33.5124
2    32.5144
3    31.4948
4    32.4748
Name: lightness, dtype: float64

In [114]:
shifed_lightness[:5]

array([    nan, 33.5124, 32.5144, 31.4948, 32.4748])

In [122]:
diff_lightness = shifed_lightness - av_lightness_per_frame

In [123]:
diff_lightness[:5]

array([-33.5124,   0.998 ,   1.0196,  -0.98  ,  -1.0376])

In [118]:
agg_df.light_diff[:5]

frame
0       NaN
1    0.9980
2    1.0196
3   -0.9800
4   -1.0376
Name: light_diff, dtype: float64

In [119]:
vid.shape

(300, 50, 50, 3)

In [120]:
lightness_per_frame.shape

(300, 2500)

In [125]:
from typing import List, Union
from numpy.typing import ArrayLike

In [194]:
def get_lightness_difference(vid: Union[str, ArrayLike], fps: int = 30,
               conversion: int = cv.COLOR_BGR2HLS) -> np.array:
    '''
    returns an numpy array of difference in mean of lightness between frames.
    the length of the array = number of video frames - 1
    '''
    if isinstance(vid, str):
        vid, fps = VideoReader.get_vid(vid, conversion)
    frames = vid.shape[0]
    print(frames)
    height = vid.shape[1]
    width = vid.shape[2]
    # creates an numpy array with the lightness values of each frame
    # the shape of the array is (frames, height*width) f.e. (300, 2500)
    lightness_per_frame = vid.reshape(-1, 3)[:, 1].reshape((-1, height*width))
    # aggregate by mean
    av_lightness_per_frame = np.mean(lightness_per_frame,axis=1)
    # shift values by 1 position down
    shifed_lightness = np.concatenate([np.zeros(1), av_lightness_per_frame[:-1]])
    # set 1st value to NaN
    shifed_lightness[0] = np.nan
    # get the difference in lightness between frames
    diff_lightness = shifed_lightness - av_lightness_per_frame
    # return all values but NaN
    return diff_lightness[1:]

In [128]:
import re

In [138]:
re.search('(\w+)', filenames[0]).group()

'hazard'

In [139]:
directory = 'videos1/'

In [195]:
data_list = []
data = {'Number':[], 'Sequence':[], 'ClassName':[]}
for i, file_name in enumerate(filenames):
    path = directory + file_name
    sq = get_lightness_difference(path)
    print(len(sq))
    class_name = re.search('(\w+)', file_name).group()
    data_list.append({'Number': i, 'Sequence': sq, 'ClassName': class_name}) 
    data['Number'].append(i)
    data['Sequence'].append(sq)
    data['ClassName'].append(class_name)

3927
3926
785
784
4561
4560
300
299
768
767
7346
7345
150
149
2304
2303
1521
1520
150
149
6314
6313
297
296
3933
3932
300
299
2375
2374
841
840
1159
1158
407
406
888
887
1290
1289


In [179]:
path = 'videos1/safe.04.mp4'
df1 = df.get_exploration_df(path)

In [180]:
df1.head(1)

,,,hue,lightness,saturation,masked_values,seconds
frame,y,x,,,,,
0,0,0,60,99,118,0,0


In [181]:
agg_df1 = df.get_aggregated_df(df1)

In [182]:
agg_df1.tail(2)

,lightness,hue,saturation,masked_values,light_diff,hue_diff,saturation_diff,mask_diff
frame,,,,,,,,
4559,251.0644,4.17,8.0192,2455,0.0000,0.00,0.0000,0.0
4560,255.0000,0.00,0.0000,2500,-3.9356,4.17,8.0192,-45.0


In [150]:
import json
import pickle

In [152]:
with open("videos1/data_list.pkl", "wb") as outfile:
    pickle.dump(data_list, outfile)
with open("videos1/data.pkl", "wb") as outfile1:
    pickle.dump(data, outfile1)

In [153]:
with open("videos1/data.pkl", "rb") as f:
    x = pickle.load(f)

In [165]:

data_for_labeling = np.array(data['Sequence'])

# create a corresponding label array 
labels = np.array(data['ClassName'])



In [167]:
data_for_labeling.shape

(20, 299)

In [166]:
labels

array(['hazard', 'hazard', 'safe', 'hazard', 'safe', 'hazard', 'hazard',
       'safe', 'hazard', 'hazard', 'hazard', 'safe', 'hidden', 'hazard',
       'hazard', 'hidden', 'hidden', 'hidden', 'hidden', 'hidden'],
      dtype='<U6')

In [177]:
from sklearn.preprocessing import LabelEncoder
# make sure that the label array is of the same data type as the numpy array
le = LabelEncoder()
num_labels = le.fit_transform(labels)
labels = num_labels.astype(data_for_labeling.dtype)

In [178]:
labels

array([0., 0., 2., 0., 2., 0., 0., 2., 0., 0., 0., 2., 1., 0., 0., 1., 1.,
       1., 1., 1.])

0 - hazard

1 - hidden

2 - safe

In [185]:
path = 'videos1/safe.04.mp4'
vid, fps = VideoReader.get_vid(path, cv.COLOR_BGR2HLS)

In [189]:
    frames = vid.shape[0]
    height = vid.shape[1]
    width = vid.shape[2]

In [190]:
lightness_per_frame = vid.reshape(-1, 3)[:, 1].reshape((-1, height*width))

In [191]:
lightness_per_frame.shape

(4561, 2500)

In [192]:
shifed_lightness = np.concatenate([np.zeros(1), av_lightness_per_frame[:-1]])

In [193]:
shifed_lightness.shape # found an error

(300,)